<a href="https://colab.research.google.com/github/sky1ove/Kaggle_HuBMAP/blob/main/Iafoss_fastai_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd /content/gdrive/My Drive/HuBMAP

/content/gdrive/My Drive/HuBMAP


In [3]:
!pip install -U fastai #-U means updates

     |████████████████████████████████| 194kB 10.8MB/s 
     |████████████████████████████████| 61kB 5.9MB/s 
     |████████████████████████████████| 776.8MB 17kB/s 
     |████████████████████████████████| 12.8MB 24.5MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
  Found existing installation: torchvision 0.9.1+cu101
    Uninstalling torchvision-0.9.1+cu101:
      Successfully uninstalled torchvision-0.9.1+cu101
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


In [ ]:
sz = 256  #the size of tiles
reduce = 4 #reduce the original images by 4 times 
MASKS = 'train.csv'
DATA = 'train/'
OUT_TRAIN = 'train.zip'
OUT_MASKS = 'masks.zip'

In [ ]:
!unzip -q masks.zip -d "/content/gdrive/My Drive/HuBMAP/image_256/mask"  #quietly

replace /content/gdrive/My Drive/HuBMAP/image_256/mask/2f6ecfcdf_0064.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
!unzip -q train.zip -d "/content/gdrive/My Drive/HuBMAP/image_256/tiles"  #quietly

In [4]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.vision.all import *
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import os
import cv2
import gc
import random
from albumentations import *
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings("ignore")

In [5]:
import sys
sys.path.insert(0,'/content/gdrive/My Drive/HuBMAP/')

In [6]:
from lovasz1 import lovasz_hinge # to import python script, need to sys.path.insert the directory first

In [ ]:
ls

HuBMAP-0.ipynb                     tiles_r4_s256_shft/
HuBMAP-20-dataset_information.csv  Tiling_1024_reduce2_Iafoss.ipynb
image_256/                         Tiling_256_Iafoss.ipynb
lovasz1.py                         Tiling_512_reduce2_Iafoss.ipynb
lovasz.py                          tiling_vgarshin.ipynb
masks_1024.zip                     train/
masks_512.zip                      train_1024.zip
masks_r4_s256_shft/                train_512.zip
masks.zip                          train.csv
sample_submission.csv              train.zip
test/                              Untitled0.ipynb


In [7]:
bs = 64
nfolds = 4
fold = 0
SEED = 2020
TRAIN = 'image_256/tiles/'
MASKS = 'image_256/mask/'
LABELS = 'train.csv'
NUM_WORKERS = 4

In [8]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    #the following line gives ~10% speedup
    #but may lead to some stochasticity in the results 
    torch.backends.cudnn.benchmark = True
    
seed_everything(SEED)

In [9]:
mean = np.array([0.63701495, 0.4709702,  0.6817423])
std = np.array([0.15978882, 0.2245109,  0.14173926])

In [10]:
os.path.join(TRAIN,'2f6ecfcdf.png')

'image_256/tiles/2f6ecfcdf.png'

In [12]:
ids = pd.read_csv(LABELS).id.values

In [13]:
ids

array(['2f6ecfcdf', '8242609fa', 'aaa6a05cc', 'cb2d976f4', 'b9a3865fc',
       'b2dc8411c', '0486052bb', 'e79de561c', '095bf7a1f', '54f2eec69',
       '4ef6695ce', '26dc41664', 'c68fe75ea', 'afa5e8098', '1e2425f28'],
      dtype=object)

In [14]:
train =True

In [16]:
kf = KFold(n_splits=nfolds,random_state=SEED,shuffle=True)

In [17]:
ids = set(ids[list(kf.split(ids))[fold][0 if train else 1]])

In [18]:
ids

{'0486052bb',
 '095bf7a1f',
 '1e2425f28',
 '26dc41664',
 '2f6ecfcdf',
 '4ef6695ce',
 'afa5e8098',
 'b2dc8411c',
 'c68fe75ea',
 'cb2d976f4',
 'e79de561c'}

In [19]:
fnames = [fname for fname in os.listdir(TRAIN) if fname.split('_')[0] in ids]

In [20]:
fnames[:5]

['1e2425f28_0117.png',
 '1e2425f28_0125.png',
 '1e2425f28_0143.png',
 '1e2425f28_0136.png',
 '1e2425f28_0123.png']

In [22]:
img = cv2.cvtColor(cv2.imread(os.path.join(TRAIN,fnames[0])), cv2.COLOR_BGR2RGB)

In [24]:
img.ndim

3

In [23]:
img.shape

(256, 256, 3)

In [25]:
mask = cv2.imread(os.path.join(MASKS,fnames[0]),cv2.IMREAD_GRAYSCALE)

In [26]:
mask.shape

(256, 256)

In [27]:
mask.ndim

2

In [28]:
np.expand_dims(mask,2).shape

(256, 256, 1)

In [29]:
def img2tensor(img,dtype:np.dtype=np.float32):
    if img.ndim==2 : img = np.expand_dims(img,2)
    img = np.transpose(img,(2,0,1))
    return torch.from_numpy(img.astype(dtype, copy=False))

In [30]:
def get_aug(p=1.0):
    return Compose([
        HorizontalFlip(),
        VerticalFlip(),
        RandomRotate90(),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=15, p=0.9, 
                         border_mode=cv2.BORDER_REFLECT),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.3),
        OneOf([
            HueSaturationValue(10,15,10),
            CLAHE(clip_limit=2),
            RandomBrightnessContrast(),            
        ], p=0.3),
    ], p=p)

In [ ]:
#original
# class HuBMAPDataset(Dataset):
#     def __init__(self, fold=fold, train=True, tfms=None):
#         ids = pd.read_csv(LABELS).id.values
#         kf = KFold(n_splits=nfolds,random_state=SEED,shuffle=True)
#         ids = set(ids[list(kf.split(ids))[fold][0 if train else 1]])
#         self.fnames = [fname for fname in os.listdir(TRAIN) if fname.split('_')[0] in ids]
#         self.train = train
#         self.tfms = tfms
        
#     def __len__(self):
#         return len(self.fnames)
    
#     def __getitem__(self, idx):
#         fname = self.fnames[idx]

#         img = cv2.cvtColor(cv2.imread(os.path.join(TRAIN,fname)), cv2.COLOR_BGR2RGB)

#         mask = cv2.imread(os.path.join(MASKS,fname),cv2.IMREAD_GRAYSCALE)

#         if self.tfms:
#             augmented = self.tfms(image=img,mask=mask)
#             img,mask = augmented['image'],augmented['mask']
#         return img2tensor((img/255.0 - mean)/std),img2tensor(mask)


In [66]:
class HuBMAPDataset(Dataset):
    def __init__(self, fold=fold, train=True, tfms=None):
        ids = pd.read_csv(LABELS).id.values
        kf = KFold(n_splits=nfolds,random_state=SEED,shuffle=True)
        ids = set(ids[list(kf.split(ids))[fold][0 if train else 1]])
        self.fnames = [fname for fname in os.listdir(TRAIN) if fname.split('_')[0] in ids]
        self.train = train
        self.tfms = tfms
        
    def __len__(self):
        return len(self.fnames)
    
    def __getitem__(self, idx):
        fname = self.fnames[idx]
        img = cv2.cvtColor(cv2.imread(os.path.join(TRAIN,fname)), cv2.COLOR_BGR2RGB)
        #img = (img/255.0 - mean)/std
        #img.astype(np.float32, copy=False)
        mask = cv2.imread(os.path.join(MASKS,fname),cv2.IMREAD_GRAYSCALE)

        #mask.astype(np.float32, copy=False)
        if self.tfms:
            #img = img.astype(np.uint8,copy=False)
            #mask = img.astype(np.uint8,copy=False)
            augmented = self.tfms(image=img,mask=mask)
            img,mask = augmented['image'],augmented['mask']
        return img2tensor((img/255.0 - mean)/std),img2tensor(mask)


In [67]:
ds = HuBMAPDataset(tfms=get_aug())
dl = DataLoader(ds,batch_size=64,shuffle=False,num_workers=4)

In [58]:
mask = cv2.imread(os.path.join(MASKS,fnames[0]),cv2.IMREAD_GRAYSCALE)

In [59]:
mask.shape

(256, 256)

In [ ]:
# ds = HuBMAPDataset(tfms=None)
# dl = DataLoader(ds,batch_size=64,shuffle=False,num_workers=0)

In [53]:
for i in dl:
  a =i
  break

In [55]:
a[1].shape

torch.Size([64, 3, 256, 256])

In [68]:
ds[1][0].shape

torch.Size([3, 256, 256])

In [37]:
ds[1][0].dtype

torch.float32

In [69]:
ds[1][1].shape

torch.Size([1, 256, 256])

In [39]:
ds[1][1].dtype

torch.float32

In [ ]:
ds = HuBMAPDataset(tfms=get_aug())
dl = DataLoader(ds,batch_size=64,shuffle=False,num_workers=NUM_WORKERS)
imgs,masks = next(iter(dl))

plt.figure(figsize=(16,16))
for i,(img,mask) in enumerate(zip(imgs,masks)):
    img = ((img.permute(1,2,0)*std + mean)*255.0).numpy().astype(np.uint8)
    plt.subplot(8,8,i+1)
    #plt.imshow(img,vmin=0,vmax=255)
    plt.imshow(mask.squeeze().numpy(), alpha=0.2)
    plt.axis('off')
    plt.subplots_adjust(wspace=None, hspace=None)
    
del ds,dl,imgs,masks

# Model

In [72]:
class FPN(nn.Module):
    def __init__(self, input_channels:list, output_channels:list):
        super().__init__()
        self.convs = nn.ModuleList(
            [nn.Sequential(nn.Conv2d(in_ch, out_ch*2, kernel_size=3, padding=1),
             nn.ReLU(inplace=True), nn.BatchNorm2d(out_ch*2),
             nn.Conv2d(out_ch*2, out_ch, kernel_size=3, padding=1))
            for in_ch, out_ch in zip(input_channels, output_channels)])
        
    def forward(self, xs:list, last_layer):
        hcs = [F.interpolate(c(x),scale_factor=2**(len(self.convs)-i),mode='bilinear') 
               for i,(c,x) in enumerate(zip(self.convs, xs))]
        hcs.append(last_layer)
        return torch.cat(hcs, dim=1)

class UnetBlock(Module):
    def __init__(self, up_in_c:int, x_in_c:int, nf:int=None, blur:bool=False,
                 self_attention:bool=False, **kwargs):
        super().__init__()
        self.shuf = PixelShuffle_ICNR(up_in_c, up_in_c//2, blur=blur, **kwargs)
        self.bn = nn.BatchNorm2d(x_in_c)
        ni = up_in_c//2 + x_in_c
        nf = nf if nf is not None else max(up_in_c//2,32)
        self.conv1 = ConvLayer(ni, nf, norm_type=None, **kwargs)
        self.conv2 = ConvLayer(nf, nf, norm_type=None,
            xtra=SelfAttention(nf) if self_attention else None, **kwargs)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, up_in:Tensor, left_in:Tensor) -> Tensor:
        s = left_in
        up_out = self.shuf(up_in)
        cat_x = self.relu(torch.cat([up_out, self.bn(s)], dim=1))
        return self.conv2(self.conv1(cat_x))
        
class _ASPPModule(nn.Module):
    def __init__(self, inplanes, planes, kernel_size, padding, dilation, groups=1):
        super().__init__()
        self.atrous_conv = nn.Conv2d(inplanes, planes, kernel_size=kernel_size,
                stride=1, padding=padding, dilation=dilation, bias=False, groups=groups)
        self.bn = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU()

        self._init_weight()

    def forward(self, x):
        x = self.atrous_conv(x)
        x = self.bn(x)

        return self.relu(x)

    def _init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                torch.nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

class ASPP(nn.Module):
    def __init__(self, inplanes=512, mid_c=256, dilations=[6, 12, 18, 24], out_c=None):
        super().__init__()
        self.aspps = [_ASPPModule(inplanes, mid_c, 1, padding=0, dilation=1)] + \
            [_ASPPModule(inplanes, mid_c, 3, padding=d, dilation=d,groups=4) for d in dilations]
        self.aspps = nn.ModuleList(self.aspps)
        self.global_pool = nn.Sequential(nn.AdaptiveMaxPool2d((1, 1)),
                        nn.Conv2d(inplanes, mid_c, 1, stride=1, bias=False),
                        nn.BatchNorm2d(mid_c), nn.ReLU())
        out_c = out_c if out_c is not None else mid_c
        self.out_conv = nn.Sequential(nn.Conv2d(mid_c*(2+len(dilations)), out_c, 1, bias=False),
                                    nn.BatchNorm2d(out_c), nn.ReLU(inplace=True))
        self.conv1 = nn.Conv2d(mid_c*(2+len(dilations)), out_c, 1, bias=False)
        self._init_weight()

    def forward(self, x):
        x0 = self.global_pool(x)
        xs = [aspp(x) for aspp in self.aspps]
        x0 = F.interpolate(x0, size=xs[0].size()[2:], mode='bilinear', align_corners=True)
        x = torch.cat([x0] + xs, dim=1)
        return self.out_conv(x)
    
    def _init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                torch.nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

In [73]:
class UneXt50(nn.Module):
    def __init__(self, stride=1, **kwargs):
        super().__init__()
        #encoder
        m = torch.hub.load('facebookresearch/semi-supervised-ImageNet1K-models',
                           'resnext50_32x4d_ssl')
        self.enc0 = nn.Sequential(m.conv1, m.bn1, nn.ReLU(inplace=True))
        self.enc1 = nn.Sequential(nn.MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1),
                            m.layer1) #256
        self.enc2 = m.layer2 #512
        self.enc3 = m.layer3 #1024
        self.enc4 = m.layer4 #2048
        #aspp with customized dilatations
        self.aspp = ASPP(2048,256,out_c=512,dilations=[stride*1,stride*2,stride*3,stride*4])
        self.drop_aspp = nn.Dropout2d(0.5)
        #decoder
        self.dec4 = UnetBlock(512,1024,256)
        self.dec3 = UnetBlock(256,512,128)
        self.dec2 = UnetBlock(128,256,64)
        self.dec1 = UnetBlock(64,64,32)
        self.fpn = FPN([512,256,128,64],[16]*4)
        self.drop = nn.Dropout2d(0.1)
        self.final_conv = ConvLayer(32+16*4, 1, ks=1, norm_type=None, act_cls=None)
        
    def forward(self, x):
        enc0 = self.enc0(x)
        enc1 = self.enc1(enc0)
        enc2 = self.enc2(enc1)
        enc3 = self.enc3(enc2)
        enc4 = self.enc4(enc3)
        enc5 = self.aspp(enc4)
        dec3 = self.dec4(self.drop_aspp(enc5),enc3)
        dec2 = self.dec3(dec3,enc2)
        dec1 = self.dec2(dec2,enc1)
        dec0 = self.dec1(dec1,enc0)
        x = self.fpn([enc5, dec3, dec2, dec1], dec0)
        x = self.final_conv(self.drop(x))
        x = F.interpolate(x,scale_factor=2,mode='bilinear')
        return x

#split the model to encoder and decoder for fast.ai
split_layers = lambda m: [list(m.enc0.parameters())+list(m.enc1.parameters())+
                list(m.enc2.parameters())+list(m.enc3.parameters())+
                list(m.enc4.parameters()),
                list(m.aspp.parameters())+list(m.dec4.parameters())+
                list(m.dec3.parameters())+list(m.dec2.parameters())+
                list(m.dec1.parameters())+list(m.fpn.parameters())+
                list(m.final_conv.parameters())]

In [74]:
def symmetric_lovasz(outputs, targets):
    return 0.5*(lovasz_hinge(outputs, targets) + lovasz_hinge(-outputs, 1.0 - targets))

In [75]:
class Dice_soft(Metric):
    def __init__(self, axis=1): 
        self.axis = axis 
    def reset(self): self.inter,self.union = 0,0
    def accumulate(self, learn):
        pred,targ = flatten_check(torch.sigmoid(learn.pred), learn.y)
        self.inter += (pred*targ).float().sum().item()
        self.union += (pred+targ).float().sum().item()
    @property
    def value(self): return 2.0 * self.inter/self.union if self.union > 0 else None
    
# dice with automatic threshold selection
class Dice_th(Metric):
    def __init__(self, ths=np.arange(0.1,0.9,0.05), axis=1): 
        self.axis = axis
        self.ths = ths
        
    def reset(self): 
        self.inter = torch.zeros(len(self.ths))
        self.union = torch.zeros(len(self.ths))
        
    def accumulate(self, learn):
        pred,targ = flatten_check(torch.sigmoid(learn.pred), learn.y)
        for i,th in enumerate(self.ths):
            p = (pred > th).float()
            self.inter[i] += (p*targ).float().sum().item()
            self.union[i] += (p+targ).float().sum().item()

    @property
    def value(self):
        dices = torch.where(self.union > 0.0, 
                2.0*self.inter/self.union, torch.zeros_like(self.union))
        return dices.max()

In [76]:
class Model_pred:
    def __init__(self, model, dl, tta:bool=True, half:bool=False):
        self.model = model
        self.dl = dl
        self.tta = tta
        self.half = half
        
    def __iter__(self):
        self.model.eval()
        name_list = self.dl.dataset.fnames
        count=0
        with torch.no_grad():
            for x,y in iter(self.dl):
                x = x.cuda()
                if self.half: x = x.half()
                p = self.model(x)
                py = torch.sigmoid(p).detach()
                if self.tta:
                    #x,y,xy flips as TTA
                    flips = [[-1],[-2],[-2,-1]]
                    for f in flips:
                        p = self.model(torch.flip(x,f))
                        p = torch.flip(p,f)
                        py += torch.sigmoid(p).detach()
                    py /= (1+len(flips))
                if y is not None and len(y.shape)==4 and py.shape != y.shape:
                    py = F.upsample(py, size=(y.shape[-2],y.shape[-1]), mode="bilinear")
                py = py.permute(0,2,3,1).float().cpu()
                batch_size = len(py)
                for i in range(batch_size):
                    taget = y[i].detach().cpu() if y is not None else None
                    yield py[i],taget,name_list[count]
                    count += 1
                    
    def __len__(self):
        return len(self.dl.dataset)
    
class Dice_th_pred(Metric):
    def __init__(self, ths=np.arange(0.1,0.9,0.01), axis=1): 
        self.axis = axis
        self.ths = ths
        self.reset()
        
    def reset(self): 
        self.inter = torch.zeros(len(self.ths))
        self.union = torch.zeros(len(self.ths))
        
    def accumulate(self,p,t):
        pred,targ = flatten_check(p, t)
        for i,th in enumerate(self.ths):
            p = (pred > th).float()
            self.inter[i] += (p*targ).float().sum().item()
            self.union[i] += (p+targ).float().sum().item()

    @property
    def value(self):
        dices = torch.where(self.union > 0.0, 2.0*self.inter/self.union, 
                            torch.zeros_like(self.union))
        return dices
    
def save_img(data,name,out):
    data = data.float().cpu().numpy()
    img = cv2.imencode('.png',(data*255).astype(np.uint8))[1]
    out.writestr(name, img)

In [ ]:
ImageDataLoaders.from_dsets??

In [80]:
learn.summary()

UneXt50 (Input shape: 64)
Layer (type)         Output Shape         Param #    Trainable 
                     64 x 64 x 128 x 128 
Conv2d                                    9408       True      
BatchNorm2d                               128        True      
ReLU                                                           
MaxPool2d                                                      
____________________________________________________________________________
                     64 x 128 x 64 x 64  
Conv2d                                    8192       True      
BatchNorm2d                               256        True      
Conv2d                                    4608       True      
BatchNorm2d                               256        True      
____________________________________________________________________________
                     64 x 256 x 64 x 64  
Conv2d                                    32768      True      
BatchNorm2d                               512        T

In [77]:
ds_t = HuBMAPDataset(fold=fold, train=True, tfms=get_aug())
ds_v = HuBMAPDataset(fold=fold, train=False)

model = UneXt50().cuda()


Downloading: "https://github.com/facebookresearch/semi-supervised-ImageNet1K-models/archive/master.zip" to /root/.cache/torch/hub/master.zip
Downloading: "https://dl.fbaipublicfiles.com/semiweaksupervision/model_files/semi_supervised_resnext50_32x4-ddb3e555.pth" to /root/.cache/torch/hub/checkpoints/semi_supervised_resnext50_32x4-ddb3e555.pth


In [78]:
data = ImageDataLoaders.from_dsets(ds_t,ds_v,bs=bs,
            num_workers=NUM_WORKERS,pin_memory=True).cuda()

In [79]:
learn = Learner(data, model, loss_func=symmetric_lovasz,
            metrics=[Dice_soft(),Dice_th()], 
            splitter=split_layers).to_fp16()

In [ ]:
learn??

In [ ]:
for param in learn.opt.param_groups[0]['params']:
    param.requires_grad = False

AttributeError: ignored

In [ ]:
dice = Dice_th_pred(np.arange(0.2,0.7,0.01))
for fold in range(nfolds):
    ds_t = HuBMAPDataset(fold=fold, train=True, tfms=get_aug())
    ds_v = HuBMAPDataset(fold=fold, train=False)
    data = ImageDataLoaders.from_dsets(ds_t,ds_v,bs=bs,
                num_workers=NUM_WORKERS,pin_memory=True).cuda()
    model = UneXt50().cuda()
    learn = Learner(data, model, loss_func=symmetric_lovasz,
                metrics=[Dice_soft(),Dice_th()], 
                splitter=split_layers).to_fp16()
    
    #start with training the head
    learn.freeze_to(-1) #doesn't work
    for param in learn.opt.param_groups[0]['params']:
        param.requires_grad = False
    learn.fit_one_cycle(6, lr_max=0.5e-2)

    #continue training full model
    learn.unfreeze()
    learn.fit_one_cycle(32, lr_max=slice(2e-4,2e-3),
        cbs=SaveModelCallback(monitor='dice_th',comp=np.greater))
    torch.save(learn.model.state_dict(),f'model_{fold}.pth')
    
    #model evaluation on val and saving the masks
    mp = Model_pred(learn.model,learn.dls.loaders[1])
    with zipfile.ZipFile('val_masks_tta.zip', 'a') as out:
        for p in progress_bar(mp):
            dice.accumulate(p[0],p[1])
            save_img(p[0],p[2],out)
    gc.collect()

Using cache found in /root/.cache/torch/hub/facebookresearch_semi-supervised-ImageNet1K-models_master


epoch,train_loss,valid_loss,dice_soft,dice_th,time


RuntimeError: ignored

In [81]:
learn.fit_one_cycle(6, lr_max=0.5e-2)

epoch,train_loss,valid_loss,dice_soft,dice_th,time


KeyboardInterrupt: ignored